In [1]:
# TO DO
#
# MVP: create a version that allows us to pass and no more

Copied from assignment 2. NEEDS WORK!!!

In [2]:
# Organize data for pytorch DataLoader
import os
import shutil
import random

DATA_DIR = '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/'
ANNOTATIONS_DIR = DATA_DIR + 'annotations/'
IMAGES_DIR = DATA_DIR + 'images/'

# New fodlers for train, test, and dev sets
TRAIN_DIR = DATA_DIR + 'train/'
DEV_DIR = DATA_DIR + 'dev/'
TEST_DIR = DATA_DIR + 'test/'

annotations = ["baby",
               "bird",
               "car",
               "clouds",
               "dog",
               "female",
               "flower",
               "male",
               "night",
               "people",
               "portrait",
               "river",
               "sea",
               "tree"]

# Create "train" folder for data
os.makedirs(TRAIN_DIR)

for annotation in annotations:
    
    # Create folder for this annotation within "train" folder
    os.makedirs(TRAIN_DIR + annotation)
    os.makedirs(DEV_DIR + annotation)
    os.makedirs(TEST_DIR + annotation)

    annotation_file = open(ANNOTATIONS_DIR + annotation + ".txt", "r")
    annotation_data = annotation_file.read()
    annotation_file.close()
    data_into_list = annotation_data.split('\n')
    image_file_names = ["im" + i + ".jpg" for i in data_into_list]

    for image_file_name in image_file_names:
        if os.path.isfile(IMAGES_DIR + image_file_name):
            division = random.randint(1, 3)
            if division == 1:
                shutil.copyfile(IMAGES_DIR + image_file_name, TRAIN_DIR + annotation + "/" + image_file_name)
            if division == 2:
                shutil.copyfile(IMAGES_DIR + image_file_name, DEV_DIR + annotation + "/" + image_file_name)
            if division == 3:
                shutil.copyfile(IMAGES_DIR + image_file_name, TEST_DIR + annotation + "/" + image_file_name)

In [3]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


#--- hyperparameters ---
N_EPOCHS = 4
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.0003
WEIGHT_DECAY = 0.0000
MOMENTUM = 0.1


#--- fixed constants ---
NUM_CLASSES = 14
NUM_CHANNELS = 3



# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.
train_transform = transforms.Compose([
                                        #transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

train_set = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
dev_set   = datasets.ImageFolder(DEV_DIR,   transform=test_transform)
test_set  = datasets.ImageFolder(TEST_DIR,  transform=test_transform)

# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, shuffle=False)


#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(NUM_CHANNELS, 20, (5,5)),
            nn.ReLU(),
            nn.MaxPool2d((2,2), stride = (2,2)),
            nn.Conv2d(20, 50, (5,5)),
            nn.ReLU(),
            nn.MaxPool2d((2,2), stride = (2,2))
        )
        self.flatten = nn.Flatten()
        self.lin = nn.Sequential(
            nn.Linear(42050, 100),
            nn.ReLU(),
            # nn.Tanh(),
            nn.Linear(100, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.lin(x)
        return x



#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)

# WRITE CODE HERE
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
loss_function = nn.NLLLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
stop_early = False

#--- training ---
for epoch in range(N_EPOCHS):
    if stop_early:
        break
    train_loss = 0
    train_correct = 0
    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # WRITE CODE HERE
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        train_correct += (pred.argmax(1) == target).type(torch.float).sum().item()

        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct / total, train_correct, total))
    
    # WRITE CODE HERE
    # Please implement early stopping here.
    # You can try different versions, simplest way is to calculate the dev error and
    # compare this with the previous dev error, stopping if the error has grown.
    cur_dev_loss = 0
    dev_correct = 0
    total = 0

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            # WRITE CODE HERE
            pred = model(data)
            loss = loss_function(pred, target)

            total += len(data)
            cur_dev_loss += loss.item()
            dev_correct += (pred.argmax(1) == target).type(torch.float).sum().item()

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = 100. * dev_correct / total
        dev_accuracies.append(current_accuracy)

        if current_loss <= dev_loss:
            dev_loss = current_loss
        else:
            stop_early = True

        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct / total, dev_correct, total))
            




#--- test ---
test_loss = 0
test_correct = 0
total = 0

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        # WRITE CODE HERE
        pred = model(data)
        loss = loss_function(pred, target)

        total += len(data)
        test_loss += loss.item()
        test_correct += (pred.argmax(1) == target).type(torch.float).sum().item()

        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct / total, test_correct, total))
print(dev_losses)
print(dev_accuracies)

/Users/hartih/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training: Epoch 1 - Batch 1/68: Loss: 2.6461 | Train Acc: 1.000% (1/100)
Training: Epoch 1 - Batch 2/68: Loss: 2.5151 | Train Acc: 10.000% (20/200)
Training: Epoch 1 - Batch 3/68: Loss: 2.3695 | Train Acc: 17.000% (51/300)
Training: Epoch 1 - Batch 4/68: Loss: 2.3722 | Train Acc: 18.000% (72/400)
Training: Epoch 1 - Batch 5/68: Loss: 2.3395 | Train Acc: 17.200% (86/500)
Training: Epoch 1 - Batch 6/68: Loss: 2.2913 | Train Acc: 21.000% (126/600)
Training: Epoch 1 - Batch 7/68: Loss: 2.2828 | Train Acc: 22.000% (154/700)
Training: Epoch 1 - Batch 8/68: Loss: 2.2884 | Train Acc: 22.625% (181/800)
Training: Epoch 1 - Batch 9/68: Loss: 2.2862 | Train Acc: 24.111% (217/900)
Training: Epoch 1 - Batch 10/68: Loss: 2.2763 | Train Acc: 24.700% (247/1000)
Training: Epoch 1 - Batch 11/68: Loss: 2.2650 | Train Acc: 25.545% (281/1100)
Training: Epoch 1 - Batch 12/68: Loss: 2.2537 | Train Acc: 26.083% (313/1200)
Training: Epoch 1 - Batch 13/68: Loss: 2.2473 | Train Acc: 26.000% (338/1300)
Training: E

Training: Epoch 2 - Batch 38/68: Loss: 1.9369 | Train Acc: 30.368% (1154/3800)
Training: Epoch 2 - Batch 39/68: Loss: 1.9324 | Train Acc: 30.667% (1196/3900)
Training: Epoch 2 - Batch 40/68: Loss: 1.9326 | Train Acc: 30.700% (1228/4000)
Training: Epoch 2 - Batch 41/68: Loss: 1.9270 | Train Acc: 30.902% (1267/4100)
Training: Epoch 2 - Batch 42/68: Loss: 1.9258 | Train Acc: 30.929% (1299/4200)
Training: Epoch 2 - Batch 43/68: Loss: 1.9268 | Train Acc: 30.837% (1326/4300)
Training: Epoch 2 - Batch 44/68: Loss: 1.9229 | Train Acc: 31.045% (1366/4400)
Training: Epoch 2 - Batch 45/68: Loss: 1.9222 | Train Acc: 31.133% (1401/4500)
Training: Epoch 2 - Batch 46/68: Loss: 1.9221 | Train Acc: 31.065% (1429/4600)
Training: Epoch 2 - Batch 47/68: Loss: 1.9241 | Train Acc: 31.000% (1457/4700)
Training: Epoch 2 - Batch 48/68: Loss: 1.9232 | Train Acc: 30.917% (1484/4800)
Training: Epoch 2 - Batch 49/68: Loss: 1.9234 | Train Acc: 31.000% (1519/4900)
Training: Epoch 2 - Batch 50/68: Loss: 1.9221 | Trai

Training: Epoch 4 - Batch 5/68: Loss: 1.7828 | Train Acc: 31.800% (159/500)
Training: Epoch 4 - Batch 6/68: Loss: 1.7832 | Train Acc: 33.667% (202/600)
Training: Epoch 4 - Batch 7/68: Loss: 1.7975 | Train Acc: 33.571% (235/700)
Training: Epoch 4 - Batch 8/68: Loss: 1.8229 | Train Acc: 32.875% (263/800)
Training: Epoch 4 - Batch 9/68: Loss: 1.8094 | Train Acc: 33.444% (301/900)
Training: Epoch 4 - Batch 10/68: Loss: 1.7936 | Train Acc: 34.400% (344/1000)
Training: Epoch 4 - Batch 11/68: Loss: 1.7995 | Train Acc: 34.000% (374/1100)
Training: Epoch 4 - Batch 12/68: Loss: 1.7917 | Train Acc: 34.583% (415/1200)
Training: Epoch 4 - Batch 13/68: Loss: 1.7870 | Train Acc: 34.462% (448/1300)
Training: Epoch 4 - Batch 14/68: Loss: 1.7741 | Train Acc: 34.357% (481/1400)
Training: Epoch 4 - Batch 15/68: Loss: 1.7669 | Train Acc: 34.267% (514/1500)
Training: Epoch 4 - Batch 16/68: Loss: 1.7612 | Train Acc: 34.688% (555/1600)
Training: Epoch 4 - Batch 17/68: Loss: 1.7654 | Train Acc: 34.529% (587/17

Evaluating: Batch 47/69: Loss: 1.9614 | Test Acc: 29.489% (1386/4700)
Evaluating: Batch 48/69: Loss: 1.9413 | Test Acc: 30.625% (1470/4800)
Evaluating: Batch 49/69: Loss: 1.9232 | Test Acc: 31.653% (1551/4900)
Evaluating: Batch 50/69: Loss: 1.9057 | Test Acc: 32.720% (1636/5000)
Evaluating: Batch 51/69: Loss: 1.8892 | Test Acc: 33.647% (1716/5100)
Evaluating: Batch 52/69: Loss: 1.8725 | Test Acc: 34.635% (1801/5200)
Evaluating: Batch 53/69: Loss: 1.8572 | Test Acc: 35.566% (1885/5300)
Evaluating: Batch 54/69: Loss: 1.8420 | Test Acc: 36.537% (1973/5400)
Evaluating: Batch 55/69: Loss: 1.8265 | Test Acc: 37.436% (2059/5500)
Evaluating: Batch 56/69: Loss: 1.8245 | Test Acc: 36.964% (2070/5600)
Evaluating: Batch 57/69: Loss: 1.8219 | Test Acc: 36.544% (2083/5700)
Evaluating: Batch 58/69: Loss: 1.8206 | Test Acc: 36.034% (2090/5800)
Evaluating: Batch 59/69: Loss: 1.8208 | Test Acc: 35.492% (2094/5900)
Evaluating: Batch 60/69: Loss: 1.8196 | Test Acc: 35.000% (2100/6000)
Evaluating: Batch 61